# Emotion Detection

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [malaya-speech/example/emotion](https://github.com/huseinzol05/malaya-speech/tree/master/example/emotion).
    
</div>

<div class="alert alert-info">

This module is language independent, so it save to use on different languages.
    
</div>

<div class="alert alert-warning">

This is an application of malaya-speech Pipeline, read more about malaya-speech Pipeline at [malaya-speech/example/pipeline](https://github.com/huseinzol05/malaya-speech/tree/master/example/pipeline).
    
</div>

### Dataset

Trained on Toronto emotional speech set (TESS) with augmented noises, https://tspace.library.utoronto.ca/handle/1807/24487

In [ ]:
!pip install malaya_speech
!pip install networkx
!pip install subprocess
!pip install graphviz


In [ ]:
import subprocess
import os
import malaya_speech
import numpy as np
from malaya_speech import Pipeline

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # to stop tensorflow form loading cuda

In [ ]:
command = "ffmpeg -y -i ./Center.mp4 -ab 160k -ac 2 -ar 44100 -vn audio.wav"
subprocess.call(command)

In [ ]:
command = "ffmpeg -y -ss 174 -i ./audio.wav -t 00:00:9 trimed.wav"
subprocess.call(command)

In [ ]:
y, sr = malaya_speech.load('./trimed.wav')
len(y), sr

In [ ]:
# just going to take 30 seconds
#y = y[:sr* 30]

### Supported emotions

In [ ]:
malaya_speech.emotion.labels

# Deep-Speaker Model

In [ ]:
deep_speaker = malaya_speech.emotion.deep_model(model ="deep-speaker",validate = False)

### How to classify emotions in an audio sample

So we are going to use VAD to help us. Instead we are going to classify as a whole sample, we chunk it into multiple small samples and classify it.

In [ ]:
vad = malaya_speech.vad.deep_model(model = 'vggvox-v2')

In [ ]:
%%time
frames = list(malaya_speech.utils.generator.frames(y, 30, sr))

In [ ]:
p = Pipeline()
pipeline = (
    p.batching(5)
    .foreach_map(vad.predict)
    .flatten()
)

In [ ]:
%%time
result = p.emit(frames)
result.keys()

In [ ]:
frames_vad = [(frame, result['flatten'][no]) for no, frame in enumerate(frames)]
grouped_vad = malaya_speech.utils.group.group_frames(frames_vad)
grouped_vad = malaya_speech.utils.group.group_frames_threshold(grouped_vad, threshold_to_stop = 0.3)

In [ ]:
p_deep_speaker = Pipeline()
pipeline = (
    p_deep_speaker.foreach_map(deep_speaker)
    .flatten()
)

In [ ]:
%%time
samples_vad = [g[0] for g in grouped_vad]
result_deep_speaker = p_deep_speaker.emit(samples_vad)
result_deep_speaker.keys()

In [ ]:
samples_vad_deep_speaker = [(frame, result_deep_speaker['flatten'][no]) for no, frame in enumerate(samples_vad)]
samples_vad_deep_speaker

### Reference

1. Toronto emotional speech set (TESS), https://tspace.library.utoronto.ca/handle/1807/24487
2. The Singaporean White Boy - The Shan and Rozz Show: EP7, https://www.youtube.com/watch?v=HylaY5e1awo&t=2s&ab_channel=Clicknetwork